In [115]:
import arcpy
import os
import sys
ws = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\TaxLots\TaxLots.gdb'
arcpy.env.workspace = ws

In [116]:
def unique_values(table, field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
    
        return sorted({row[0] for row in cursor if row[0] != None})

In [117]:
def pubFind(searchTerms, myValues):
    pubLand = []
    myValues = [value.upper() for value in myValues]
    
    for owner in myValues:
        for term in searchTerms:
            if term in owner:
                pubLand.append(owner)
    return pubLand

In [118]:
def removeTerms(OgList, removeList):
    for item in removeList:
        if item in OgList:
            OgList.remove(item)
    return OgList

In [119]:
def parseLandOwners(pubOwnList, parseTermsList, returnList):
    returnList = []
    for owner in pubOwnList:
        for term in parseTermsList:
            if term in owner:
                returnList.append(owner)
    return returnList

In [120]:
pubLands = 'publicLandsFloodplain'
keithLyr = 'pub_pvt_exportpub_wm_Project'

In [121]:
#keithUnique = unique_values(keithLyr, 'pub_pvt')

In [122]:
#print(keithUnique)

In [123]:
cityTerms = ['CITY', 'PORT']
cityParkTerms = ['PARK', 'CITY']
countyTerms = ['CO', 'MULTNOMAH', 'CLATSOP', 'CLARK', 'WAHKIAKUM', 'COLUMBIA', 'PACIFIC', 'COWLITZ', 'SKAMANIA', 'SCHOOL']
fedTerms = ['US', 'USA', 'U S', 'UNITED', 'FED']
fedFwTerms = ['US', 'USA', 'U S', 'UNITED', 'FED', 'FISH', 'WILDLIFE', 'FW', 'F & W', 'F AND W']
fedForestTerms = ['US', 'USA', 'U S', 'UNITED', 'FED', 'FOREST']
metroTerms = ['MET']
stTerms = ['STATE', 'OR', 'WA']
stDnrTerms = ['STATE', 'OR', 'WA', 'NATURAL', 'RESOURCES']
stDslTerms = ['STATE', 'OR', 'WA', 'LANDS']
stFwTerms = ['STATE', 'OR', 'WA', 'FISH', 'WILDLIFE']
stForestTerms = ['STATE', 'OR', 'WA', 'FOREST']
stParkTerms = ['STATE', 'OR', 'WA', 'PARK']

termsList = [cityTerms, cityParkTerms, countyTerms, fedTerms, fedFwTerms, fedForestTerms, metroTerms, stTerms, stDnrTerms, stDslTerms, 
            stFwTerms, stForestTerms, stParkTerms]

In [124]:
cityList = 'cityList'
cityParkList = 'cityParkList'
countyList = 'countyList'
fedList = 'fedList'
fedFwList ='fedFwList'
fedForestList = 'fedForestList'
metroList = 'metroList'
stList = 'stList'
stDnrList = 'stDnrList'
stDslList = 'stDslList'
stFwList = 'stFwList'
stForestList = 'stForestList'
stParkList = 'stParkList'

parsedOwnersList = [cityList, cityParkList, countyList, fedList, fedFwList, fedForestList, metroList, stList, stDnrList, 
                    stDslList, stFwList, stForestList, stParkList]

In [125]:
pubOwners = unique_values(pubLands, 'Land_Owner')

In [126]:
landClassDict = {}
for parseList, returnedList in zip(termsList, parsedOwnersList):
     
    landClassDict[returnedList] = parseLandOwners(pubOwners, parseList, returnedList)
   

In [127]:
landClassDict['cityList'] = [i for i in landClassDict['cityList'] if not ('PARK' in i or 'TRANSPORT' in i or 'STATE' in i or 
                                                                         'TRASPORT' in i)]

In [128]:
landClassDict['cityList'] = landClassDict['cityList'] + ['HOUSING AUTHORITY OF VANCOUVER', 'HOUSING AUTHORITY VANCOUVER', 'PORTLAND CITY OF & OREGON STATE OF',
                                                        'PORTLAND CITY OF & TRI-COUNTY METROPOLITAN TRANSPORTATION DISTRICT OF OREGON', 
                                                         'PORTLAND CITY OF & OREGON STATE OF'] 

In [129]:
landClassDict['cityParkList'] =  [i for i in landClassDict['cityParkList'] if ('PARK' in i)]
landClassDict['cityParkList'] = [i for i in landClassDict['cityParkList'] if not ('STATE' in i or 'COUNTY' in i or 
                                                                                  'OREGON' in i)]


In [130]:
landClassDict['countyList'] = [i for i in landClassDict['countyList'] if not ('CITY' in i or 'STATE' in i)]

In [131]:
removeList = ['HOUSING AUTHORITY VANCOUVER', 'METRO (COLUMBIA PIONEER CEMETERY', 'METRO (COLUMBIA PIONEER CEMETERY' ]

In [132]:
landClassDict['countyList'] = removeTerms(landClassDict['countyList'], removeList)

In [133]:
landClassDict['fedList'] = [i for i in landClassDict['fedList'] if not ('COUNTY' in i or 'VANCOUVER' in i or 'FISH' in i)]

In [134]:
landClassDict['fedFwList'] = [i for i in landClassDict['fedFwList'] if not ('STATE' in i or 'CITY' in i or 'OR' in i)]

In [135]:
landClassDict['fedFwList'] = [i for i in landClassDict['fedFwList'] if ('FISH' in i)]

In [136]:
landClassDict['fedForestList'] = []

In [137]:
landClassDict['metroList'] = [i for i in landClassDict['metroList'] if not ('CEMETERY' in i or 'COUNTY' in i )]

In [138]:
landClassDict['stList'] = [i for i in landClassDict['stList'] if not ('CITY' in i or 'COUNTY' in i or 'ASTORIA' in i or 'WARRENTON' in i or
                                                                      'FISH' in i or 'RESOURCES' in i or 'STATE LANDS'
                                                                     in i or 'FOREST' in i or 'PARK' in i or 'UNITED' in i or
                                                                     'DNR' in i or 'WASTEWATER' in i or 'COLUMBIA SOIL AND WATER CONSERVATION DIS' in i 
                                                                      or 'HOUSING AUTHORITY VANCOUVER' in i or 'WILDLIFE' in i or 'WASHOUGAL' in i
                                                                     or 'PORT OF' in i)]

In [139]:
dnrUpdateList = [i for i in landClassDict['stDnrList'] if ('RESOURCES' in i or 'DNR' in i)]

In [140]:
dnrUpdateList = [i for i in dnrUpdateList if not ( i == 'US NATURAL RESOURCES')]

In [141]:
landClassDict['stDnrList'] = dnrUpdateList

In [142]:
dslUpdateList = [i for i in landClassDict['stDslList'] if ('LAND' in i or 'DSL' in i)]

In [143]:
dslUpdateList = [i for i in dslUpdateList if not ('PORTLAND' in i or 'WOODLAND' in i or 'LEGACY' in i)]

In [144]:
landClassDict['stDslList'] = dslUpdateList

In [145]:
stFwUpdateList = [i for i in landClassDict['stFwList'] if ('FISH' in i or 'WILDLIFE' in i or 'FW' in i)]

In [146]:
stFwUpdateList = [i for i in stFwUpdateList if not (i == 'DEPART OF INTERIOR US FISH & WILDLIFE SE' or 
                                                    i == 'RIDGEFIELD WILDLIFE REFUGE & CITY OF RIDGEFIELD' or 'USA' in i)]

In [147]:
landClassDict['stFwList'] = stFwUpdateList

In [148]:
stForestUpdateList = [i for i in landClassDict['stForestList'] if ('FOREST' in i)]

In [149]:
landClassDict['stForestList'] = stForestUpdateList

In [150]:
stParkUpdateList = [i for i in landClassDict['stParkList'] if ('PARK' in i or 'RECREATION' in i or 'P&R' in i or 'PR' in i)]

In [151]:
stParkUpdateList = [i for i in stParkUpdateList if not ('CITY' in i or 'COUNTY' in i or 'PORTLAND' in i or 'PROGRAM' in i or
                                                    i == 'PUBLIC PARK')]

In [152]:
landClassDict['stParkList'] = stParkUpdateList

In [153]:
landClassDict.keys()

dict_keys(['cityList', 'cityParkList', 'countyList', 'fedList', 'fedFwList', 'fedForestList', 'metroList', 'stList', 'stDnrList', 'stDslList', 'stFwList', 'stForestList', 'stParkList'])

In [154]:
for key, value in landClassDict.items():
    key = key.replace("List", '').replace('st', 'STATE ').replace('city', 'CITY ').replace('county', 'COUNTY ').replace('fed', 'FED ')

In [155]:
landClassDict.keys()

dict_keys(['cityList', 'cityParkList', 'countyList', 'fedList', 'fedFwList', 'fedForestList', 'metroList', 'stList', 'stDnrList', 'stDslList', 'stFwList', 'stForestList', 'stParkList'])

In [156]:
arcpy.AddField_management(pubLands, 'OwnerClass', 'TEXT', field_length = 100)

<Result 'C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb\\publicLandsFloodplain'>

In [157]:
with arcpy.da.UpdateCursor(pubLands, ['Land_Owner', 'OwnerClass']) as cursor:
    for row in cursor:
        for key, value in landClassDict.items():
            #for item in value:
            if row[0] in value:
                key = (key.replace('List','').replace('st', 'STATE ').replace('county', 'COUNTY').replace('city', 'CITY ').replace('fed', 'FEDERAL '))
                key = key.upper().strip().replace('FORESTATE', 'FOREST')
                row[1] = key
               
        cursor.updateRow(row)

In [158]:
#with arcpy.da.UpdateCursor('publicSinceLastUpdate', ['LandOwner', 'OwnerClass']) as cursor:
    #for row in cursor:
       # for key, value in landClassDict.items():
            #for item in value:
            #if row[0] in value:
                #key = (key.replace('List','').replace('st', 'STATE ').replace('county', 'COUNTY').replace('city', 'CITY ').replace('fed', 'FEDERAL '))
                #key = key.upper().strip().replace('FORESTATE', 'FOREST')
                #row[1] = key
               
        #cursor.updateRow(row)

In [159]:
classes = unique_values(pubLands, 'OwnerClass')

In [160]:
print(classes)

['CITY', 'CITY PARK', 'COUNTY', 'FEDERAL', 'FEDERAL FW', 'METRO', 'STATE', 'STATE DNR', 'STATE DSL', 'STATE FOREST', 'STATE FW', 'STATE PARK']


In [161]:
classDict = {'CITY': ['CITY', 'CITY PARK'], 'COUNTY': ['COUNTY'], 'FEDERAL': ['FEDERAL', 'FEDERAL FW'], 'METRO': ['METRO'],
            'STATE':['STATE', 'STATE FOREST', 'STATE FW', 'STATE PARK', 'STATE DSL', 'STATE DNR']}

In [162]:
fcUpdate = [r'OregonLambert\PublicSinceLastUpdate',  
            r'SphericalMercator\PublicSinceLastUpdate_WGS']

In [163]:
fcUpdate2 = []
for fc in fcUpdate:
    fcUpdate2.append(os.path.join(ws, fc))
    

In [164]:
fcUpdate = fcUpdate2

In [165]:
fcUpdate

['C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb\\OregonLambert\\PublicSinceLastUpdate',
 'C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb\\SphericalMercator\\PublicSinceLastUpdate_WGS']

In [166]:
arcpy.AddField_management(pubLands, 'OwnerSuperClass', 'TEXT', field_length = 100)

<Result 'C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb\\publicLandsFloodplain'>

In [167]:
 with arcpy.da.UpdateCursor(pubLands, ['OwnerClass', 'OwnerSuperClass']) as cursor:
        for row in cursor:
            for key, value in classDict.items():
            
                if row[0] in value:
            
                    row[1] = key
               
            cursor.updateRow(row)

# Next Script is Priv/Public Layers